# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выбрать данные по странам своей группы (совместно): <br>
    3530203_70101: Germany, Netherlands <br>
    3530203_70102: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_70301: Sweden, Finland, Norway, Denmark, Iceland<br>
    3530903_70302: Spain, Portugal, France, Italy, Belgium<br>
    
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

In [3]:
import pandas as pd
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth, fpmax

pd.options.display.max_rows = 100

In [4]:
data = pd.read_csv("lastfm.csv")
data.head()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


In [3]:
#pip install mlxtend

### Скорректируем данные

In [5]:
data['country'] = data['country'].str.strip(' ')
data['country'] = data['country'].str.capitalize()

In [6]:
data['artist'] = data['artist'].str.strip(' ')
data['artist'] = data['artist'].str.lower()

### Возьмем только Germany, Netherlands

In [7]:
data = data[data['country'].isin(['Germany','Netherlands'])]
# Netherlands Antilles - don't include

In [8]:
data['country'].value_counts()

Germany        24251
Netherlands     9673
Name: country, dtype: int64

### Проверим пустые и неизвестные значения

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33924 entries, 0 to 289615
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user     33924 non-null  int64 
 1   artist   33924 non-null  object
 2   sex      33924 non-null  object
 3   country  33924 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.3+ MB


In [10]:
data['sex'].unique()

array(['f', 'm'], dtype=object)

In [11]:
data['country'].unique()

array(['Germany', 'Netherlands'], dtype=object)

In [12]:
data[(data['artist'] == '[unknown]')].count()

user       95
artist     95
sex        95
country    95
dtype: int64

In [13]:
data = data[(data['artist'] != '[unknown]')]

In [15]:
data[(data['artist'] == '[unknown]')].count()

user       0
artist     0
sex        0
country    0
dtype: int64

### Проверим данные по артистам

In [14]:
data['artist'].value_counts().describe()

count    1001.000000
mean       33.795205
std        35.136747
min         1.000000
25%        13.000000
50%        22.000000
75%        41.000000
max       296.000000
Name: artist, dtype: float64

In [15]:
### Кажется, что артисты с малым количеством слушателей не явлются релевантными
### при поиске паттернов. Возьмем группы со значением больше 10

In [18]:
data = data.groupby(['artist']).filter(lambda x: x['artist'].count() > 10.)

In [19]:
data['artist'].value_counts().describe()

count    826.000000
mean      39.480630
std       36.195868
min       11.000000
25%       17.000000
50%       27.000000
75%       47.000000
max      296.000000
Name: artist, dtype: float64

In [20]:
### Проверим данные по пользователям

In [20]:
data['user'].value_counts().describe()

count    1722.000000
mean       18.937863
std        10.127036
min         1.000000
25%        11.000000
50%        19.000000
75%        26.000000
max        53.000000
Name: user, dtype: float64

In [42]:
### Кажется, что пользователи с 1 или 2 числом групп не явлются релевантными
### при поиске паттернов. Возьмем пользователей со значением больше 3

In [21]:
data = data.groupby(['user']).filter(lambda x: x['user'].count() >= 3.)

In [22]:
data['user'].value_counts().describe()

count    1671.000000
mean       19.470975
std         9.802002
min         3.000000
25%        11.000000
50%        19.000000
75%        27.000000
max        53.000000
Name: user, dtype: float64

In [23]:
data.head()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


## Поиск правил

In [24]:
data_by_user = data.groupby(['user', 'country', 'sex'])['artist'].apply(lambda x: ','.join(x).lower().strip()).reset_index()
data_by_user

,user,country,sex,artist
0,1,Germany,f,"red hot chili peppers,the black dahlia murder,..."
1,31,Netherlands,f,"jamiroquai,tori amos,jazzanova,radiohead,st. g..."
2,33,Germany,f,"death cab for cutie,tegan and sara,kimya dawso..."
3,42,Germany,f,"soundtrack,groove coverage,avril lavigne,the r..."
4,51,Germany,f,"kate nash,arctic monkeys,lykke li,adele,tegan ..."
...,...,...,...,...
1666,19662,Germany,m,"blind guardian,in extremo,subway to sally,finn..."
1667,19672,Germany,m,"system of a down,björk,portishead,the prodigy,..."
1668,19681,Netherlands,m,"the strokes,muse,led zeppelin,snow patrol,radi..."
1669,19685,Netherlands,m,"philip glass,rammstein,pink floyd,moby,blank &..."


In [25]:
#dummy encoding
dummy_data_by_user = data_by_user['artist'].str.get_dummies(',')
dummy_data_by_user = pd.concat([data_by_user, dummy_data_by_user], axis = 1)
dummy_data_by_user

,user,country,sex,artist,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,36 crazyfists,44,...,venetian snares,vnv nation,volbeat,weezer,within temptation,wolfgang amadeus mozart,yann tiersen,yeah yeah yeahs,yellowcard,zero 7
0,1,Germany,f,"red hot chili peppers,the black dahlia murder,...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,31,Netherlands,f,"jamiroquai,tori amos,jazzanova,radiohead,st. g...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,33,Germany,f,"death cab for cutie,tegan and sara,kimya dawso...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,42,Germany,f,"soundtrack,groove coverage,avril lavigne,the r...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,51,Germany,f,"kate nash,arctic monkeys,lykke li,adele,tegan ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1666,19662,Germany,m,"blind guardian,in extremo,subway to sally,finn...",0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1667,19672,Germany,m,"system of a down,björk,portishead,the prodigy,...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1668,19681,Netherlands,m,"the strokes,muse,led zeppelin,snow patrol,radi...",0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1669,19685,Netherlands,m,"philip glass,rammstein,pink floyd,moby,blank &...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
frequent_itemsets_user = apriori(dummy_data_by_user.drop(columns=['user','country', 'sex', 'artist']), 
                            min_support=0.015, use_colnames=True)

frequent_itemsets_user

,support,itemsets
0,0.018552,(2pac)
1,0.059246,(3 doors down)
2,0.041891,(30 seconds to mars)
3,0.020347,(36 crazyfists)
4,0.020946,(50 cent)
...,...,...
996,0.017355,"(rammstein, koяn, system of a down)"
997,0.015560,"(slipknot, koяn, system of a down)"
998,0.016756,"(rammstein, metallica, system of a down)"
999,0.015560,"(slipknot, rammstein, system of a down)"


## Правило 1

In [37]:
rules_by_user = association_rules(frequent_itemsets_user, metric="support", min_threshold=0.015)

In [40]:
rules_by_user[ 
    (rules_by_user['conviction'] < 1) 
    ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
382,(nirvana),(coldplay),0.091562,0.177139,0.016158,0.176471,0.996224,-0.000061,0.999188
383,(coldplay),(nirvana),0.177139,0.091562,0.016158,0.091216,0.996224,-0.000061,0.999620
398,(rammstein),(coldplay),0.136445,0.177139,0.020946,0.153509,0.866599,-0.003224,0.972084
399,(coldplay),(rammstein),0.177139,0.136445,0.020946,0.118243,0.866599,-0.003224,0.979357


"Антиправила" с lift < 1   
Такие рекомендации делать не стоит

## Правило 2

In [41]:
rules_by_user[ 
    (rules_by_user['confidence'] > 0.2) & 
    (rules_by_user['lift'] > 1) &
    (rules_by_user['consequents'] == {'maxïmo park'}) 
    ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
98,(arctic monkeys),(maxïmo park),0.102932,0.054458,0.026930,0.261628,4.804178,0.021324,1.280576
258,(bloc party),(maxïmo park),0.089168,0.054458,0.022741,0.255034,4.683089,0.017885,1.269241
836,(mando diao),(maxïmo park),0.067624,0.054458,0.015560,0.230088,4.225032,0.011877,1.228117
862,(the kooks),(maxïmo park),0.097546,0.054458,0.021544,0.220859,4.055552,0.016232,1.213569


Не самая популярная группа maxïmo park    
Можно для продвижения рекомендовать ее пользователям, слушающим arctic monkeys, bloc party, mando diao, the kooks

## Правило 3

In [43]:
frequent_itemsets_fp_user = fpgrowth(dummy_data_by_user.drop(columns=['user','country', 'sex', 'artist']),
                                     min_support=0.01, use_colnames=True)

In [46]:
rules_fp_user = association_rules(frequent_itemsets_fp_user, metric="confidence", min_threshold=0.1)
rules_fp_user

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(red hot chili peppers),(coldplay),0.159785,0.177139,0.048474,0.303371,1.712610,0.020170,1.181203
1,(coldplay),(red hot chili peppers),0.177139,0.159785,0.048474,0.273649,1.712610,0.020170,1.156762
2,(red hot chili peppers),(the killers),0.159785,0.108318,0.021544,0.134831,1.244770,0.004236,1.030645
3,(the killers),(red hot chili peppers),0.108318,0.159785,0.021544,0.198895,1.244770,0.004236,1.048821
4,(the killers),(die ärzte),0.108318,0.147816,0.022142,0.204420,1.382938,0.006131,1.071148
...,...,...,...,...,...,...,...,...,...
4561,(a perfect circle),(system of a down),0.029922,0.123878,0.011370,0.380000,3.067536,0.007664,1.413100
4562,(a perfect circle),(incubus),0.029922,0.078995,0.010772,0.360000,4.557273,0.008408,1.439071
4563,(incubus),(a perfect circle),0.078995,0.029922,0.010772,0.136364,4.557273,0.008408,1.123248
4564,(a perfect circle),(radiohead),0.029922,0.121484,0.010174,0.340000,2.798719,0.006538,1.331085


In [100]:
rules_fp_user[ 
    (rules_fp_user['lift'] < 1)  &
    (rules_fp_user['antecedents'] == {'the beatles'}) &
    (rules_fp_user['consequents'] == {'rammstein'})
].reset_index()

,index,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,2055,(the beatles),(rammstein),0.127469,0.136445,0.013764,0.107981,0.791389,-0.003628,0.96809


In [107]:
rules_fp_user[ 
    (rules_fp_user['lift'] > 1) &
    (((rules_fp_user['antecedents'] == {'the beatles'}) & (rules_fp_user['consequents'] == {'metallica'})) |
    ((rules_fp_user['antecedents'] == {'metallica'}) & (rules_fp_user['consequents'] == {'the beatles'})))
    ].reset_index()

,index,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,2256,(metallica),(the beatles),0.113704,0.127469,0.017355,0.152632,1.197405,0.002861,1.029695
1,2257,(the beatles),(metallica),0.127469,0.113704,0.017355,0.136150,1.197405,0.002861,1.025984


<ВЫВОД>

## Правило 4

In [122]:
rules_fp_user["antecedent_len"] = rules_fp_user["antecedents"].apply(lambda x: len(x))

rules_fp_user[ 
    (rules_fp_user['lift'] > 5) &
    (rules_fp_user['antecedent_len'] > 2)
].reset_index()

,index,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,1928,"(slipknot, koяn, disturbed)",(system of a down),0.013166,0.123878,0.010174,0.772727,6.237813,0.008543,3.854937,3
1,1929,"(slipknot, koяn, system of a down)",(disturbed),0.015560,0.062238,0.010174,0.653846,10.505547,0.009205,2.709090,3
2,1930,"(slipknot, disturbed, system of a down)",(koяn),0.013166,0.071215,0.010174,0.772727,10.850649,0.009236,4.086655,3
3,1931,"(koяn, disturbed, system of a down)",(slipknot),0.014961,0.055655,0.010174,0.680000,12.218065,0.009341,2.951077,3
4,1969,"(slipknot, rammstein, koяn)",(system of a down),0.012567,0.123878,0.010174,0.809524,6.534852,0.008617,4.599641,3
5,1970,"(slipknot, rammstein, system of a down)",(koяn),0.015560,0.071215,0.010174,0.653846,9.181319,0.009065,2.683157,3
6,1972,"(rammstein, koяn, system of a down)",(slipknot),0.017355,0.055655,0.010174,0.586207,10.532814,0.009208,2.282166,3


## Правило 5

In [131]:
rules_fp_user[ 
    (rules_fp_user['lift'] > 1) &
    (rules_fp_user['antecedents'] == {'timbaland'})
].reset_index()

,index,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,2465,(timbaland),(rihanna),0.038899,0.056852,0.015560,0.400000,7.035789,0.013348,1.571913,1
1,2467,(timbaland),(kanye west),0.038899,0.059844,0.015560,0.400000,6.684000,0.013232,1.566926,1
2,2469,(timbaland),(justin timberlake),0.038899,0.041293,0.014961,0.384615,9.314381,0.013355,1.557899,1
3,2470,(timbaland),(linkin park),0.038899,0.146619,0.011969,0.307692,2.098587,0.006266,1.232662,1
